In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.layers import InputLayer, Dense, Dropout

import matplotlib.pyplot as plt
import math

from time import time


In [2]:
df = pd.read_csv("/data/kddcup.data", header=None)

In [3]:
columns = [f.split(":")[0] for f in """
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: continuous.
dst_host_same_srv_rate: continuous.
dst_host_diff_srv_rate: continuous.
dst_host_same_src_port_rate: continuous.
dst_host_srv_diff_host_rate: continuous.
dst_host_serror_rate: continuous.
dst_host_srv_serror_rate: continuous.
dst_host_rerror_rate: continuous.
dst_host_srv_rerror_rate: continuous.
""".split("\n") if len(f)>0]

columns.append("Category")
print(columns)

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Category']


In [4]:
df.columns = columns

In [5]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Category
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [6]:
df.drop(columns=["num_outbound_cmds"], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 41 columns):
duration                       int64
protocol_type                  object
service                        object
flag                           object
src_bytes                      int64
dst_bytes                      int64
land                           int64
wrong_fragment                 int64
urgent                         int64
hot                            int64
num_failed_logins              int64
logged_in                      int64
num_compromised                int64
root_shell                     int64
su_attempted                   int64
num_root                       int64
num_file_creations             int64
num_shells                     int64
num_access_files               int64
is_host_login                  int64
is_guest_login                 int64
count                          int64
srv_count                      int64
serror_rate                    fl

In [8]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Category
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [9]:
df_numeric = df.select_dtypes(include=[np.float64, np.int64])
df_numeric.shape

(4898431, 37)

In [11]:
training_size = int(df.shape[0] * 0.7)

In [12]:
feature_mean = df_numeric.values[:training_size].mean(axis = 0)
feature_std = df_numeric.values[:training_size].std(axis = 0)

In [13]:
from keras.layers import LSTM, Dense

In [28]:
n_x = 37# Dimension of each transactional record
sequence_length = 10 # number of time steps that we look into to make prediction

model = keras.Sequential()
model.add(InputLayer((sequence_length, n_x)))
model.add(LSTM(10))
model.add(Dropout(0.8))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 37)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                1920      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,931
Trainable params: 1,931
Non-trainable params: 0
_________________________________________________________________


In [15]:
pd.Series(df.Category == "normal.").value_counts()

False    3925650
True      972781
Name: Category, dtype: int64

In [16]:
df_num = df

In [17]:
num_columns = df_numeric.columns
num_columns

Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login',
       'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
       'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'],
      dtype='object')

In [64]:
df_train = df.iloc[:training_size, :]
df_test = df.iloc[training_size:, :]

In [19]:
pd.options.display.max_columns = 1000

In [20]:
df_train.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06,3.428901e+06
mean,1.798946e+01,1.336918e+03,1.438338e+03,4.957857e-06,3.858379e-04,7.290966e-06,1.484645e-02,4.112105e-05,1.762547e-01,9.721191e-03,8.778323e-05,3.995449e-05,1.470529e-02,1.266295e-03,8.457520e-05,1.073522e-03,2.916386e-07,9.624075e-04,3.590027e+02,3.367904e+02,1.210919e-01,1.211769e-01,1.769130e-02,1.774800e-02,8.816183e-01,1.436075e-02,3.370236e-02,2.285571e+02,2.148149e+02,8.554925e-01,1.907444e-02,6.781845e-01,7.311792e-03,1.211629e-01,1.209890e-01,1.787890e-02,1.766435e-02
std,5.524748e+02,3.796818e+05,7.709141e+05,2.226619e-03,3.276238e-02,4.148088e-03,5.147092e-01,8.383510e-03,3.810368e-01,4.518491e+00,9.368861e-03,8.383516e-03,4.594144e+00,1.244340e-01,9.322081e-03,3.639156e-02,5.400358e-04,3.100777e-02,2.162727e+02,2.380757e+02,3.253301e-01,3.259454e-01,1.297890e-01,1.309576e-01,3.111246e-01,8.070478e-02,1.513811e-01,6.941384e+01,8.754385e+01,3.338119e-01,8.883612e-02,4.579518e-01,4.097323e-02,3.252498e-01,3.258422e-01,1.276196e-01,1.286350e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.550000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.220000e+02,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.550000e+02,2.550000e+02,1.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,1.032000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.110000e+02,5.110000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.550000e+02,2.550000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,1.032000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.110000e+02,5.110000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.550000e+02,2.550000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.832900e+04,6.933756e+08,1.309937e+0

In [21]:
feature_std

array([5.52474763e+02, 3.79681720e+05, 7.70913959e+05, 2.22661900e-03,
       3.27623707e-02, 4.14808705e-03, 5.14709113e-01, 8.38350879e-03,
       3.81036739e-01, 4.51849039e+00, 9.36885919e-03, 8.38351443e-03,
       4.59414297e+00, 1.24434017e-01, 9.32207915e-03, 3.63915531e-02,
       5.40035691e-04, 3.10077613e-02, 2.16272661e+02, 2.38075635e+02,
       3.25330051e-01, 3.25945337e-01, 1.29788998e-01, 1.30957597e-01,
       3.11124558e-01, 8.07047659e-02, 1.51381121e-01, 6.94138253e+01,
       8.75438367e+01, 3.33811862e-01, 8.88361104e-02, 4.57951771e-01,
       4.09732266e-02, 3.25249721e-01, 3.25842135e-01, 1.27619624e-01,
       1.28634967e-01])

In [22]:
[0, 1, 2, 3, 4, 5, 6, 7, 8][2:8], [0, 1, 2, 3, 4, 5, 6, 7, 8][8] 

([2, 3, 4, 5, 6, 7], 8)

In [23]:
df_train.shape[0]//128

26788

# Batching of records

```
0, 1, 2, 3, ... 100 - these are the index of the record. Each record of dim 37
```
Goal is to predict the label for record i

Logistic regression
- to do this we look at features of only record i

In RNN
- to do this we look at the features of record, i, i - 1, i - 2, ... i - sequence_len



```


Batch size: 16, and sequence length: 10


Batch 0: 
    0: X: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] y: [9]
    1: X: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] y: [10]
    ...
    15: X: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24] y: [24]

Batch 1: 
    0: X: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25], y: [25]
    1: X: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26], y: [26]
    ...
    15: X: [31, 32, 33, 34, 35, 36, 37, 38, 39, 40], y: [40]
....

Total number of batches: (100 - 10)/16 [(total_count - sequence_len)/batch_size]

Last Batch:
    0: X: [80, 81, 82, 83, 84, 85, 86, 87, 88, 89], y: [89]
    ...
    9: X: [89, 90, 91, 92, 93, 94, 95, 96, 97, 98], y: [98]
    10: X: [90, 91, 92, 93, 94, 95, 96, 97, 98, 99], y: [99]

```
Last batch has 11 ... fewer than the batch size



In [102]:
def generator(df, batch_size = 128, sequence_length = 10):
    X = df[num_columns].values
    X = (X - feature_mean)/feature_std
    y = np.where(df.Category == "normal.", 0, 1)
    
    size = X.shape[0]
    batch_start = sequence_length 
    while True:
        X_batch = []
        y_batch = []
        # For each batch
        for i in np.arange(batch_start, batch_start + batch_size):
            if i <= len(y):
                X_batch.append(X[i-sequence_length:i])
                y_batch.append(y[i-1])
        X_batch = np.array(X_batch)
        y_batch = np.array(y_batch)
        #print("\n===X: ", X_batch.shape, "y: ", y_batch.shape, "\n")
        if len(X_batch.shape) == 3:
            yield X_batch, y_batch
        batch_start = batch_start + batch_size
        if batch_start >= size:
            batch_start = sequence_length-1
            #print("\nEnd of the epoch\n")

count = 0
df = df_train.iloc[:100, :]
for batch in generator(df, 16):
    print(batch[0].shape)
    count += 1
    if count > (df.shape[0] - sequence_length)//16:
        break

(16, 10, 37)
(16, 10, 37)
(16, 10, 37)
(16, 10, 37)
(16, 10, 37)
(11, 10, 37)


In [103]:
tbaord = keras.callbacks.TensorBoard(log_dir="/tmp/tf/logs/%d" % time())
df = df_train #.iloc[:100, :]
batch_size = 128
batch_count = math.floor((df.shape[0] - sequence_length)/batch_size)
print(batch_count)
model.fit_generator(generator(df, batch_size)
                    , steps_per_epoch=batch_count
                    , epochs=2
                    , verbose=1
                    , callbacks=[tbaord])

26788
Epoch 1/2
26788/26788 [==============================] - 368s 14ms/step - loss: 0.1131 - acc: 0.9739
Epoch 2/2
26788/26788 [==============================] - 362s 14ms/step - loss: 0.1128 - acc: 0.9742


In [108]:
batch_size = 256
df = df_test

batch_count = (df.shape[0] - sequence_length)//batch_size
model.evaluate_generator(generator(df, batch_size), batch_count)

[0.01926604228812845, 0.9984449858449478]